In [17]:

import plotly.graph_objs as go
import plotly.io as pio
import pandas as pd

In [3]:
df = pd.read_csv('/Users/joaosequeira/udacity_joao/atp_analysis/atp_tennis.csv')


In [4]:
df.head(10)

,Tournament,Date,Series,Court,Surface,Round,Best of,Player_1,Player_2,Winner,Rank_1,Rank_2,Pts_1,Pts_2,Odd_1,Odd_2,score
0,Brisbane International,2012-12-31,ATP250,Outdoor,Hard,1st Round,3,Mayer F.,Giraldo S.,Mayer F.,28,57,1215,778,1.36,3.00,6-4 6-4
1,Brisbane International,2012-12-31,ATP250,Outdoor,Hard,1st Round,3,Benneteau J.,Nieminen J.,Nieminen J.,35,41,1075,927,2.20,1.61,3-6 6-2 1-6
2,Brisbane International,2012-12-31,ATP250,Outdoor,Hard,1st Round,3,Nishikori K.,Matosevic M.,Nishikori K.,19,49,1830,845,1.25,3.75,7-5 6-2
3,Brisbane International,2012-12-31,ATP250,Outdoor,Hard,1st Round,3,Mitchell B.,Baghdatis M.,Baghdatis M.,326,36,137,1070,9.00,1.07,4-6 4-6
4,Brisbane International,2013-01-01,ATP250,Outdoor,Hard,1st Round,3,Istomin D.,Klizan M.,Istomin D.,43,30,897,1175,1.90,1.80,6-1 6-2
5,Brisbane International,2013-01-01,ATP250,Outdoor,Hard,1st Round,3,Ito T.,Millman J.,Millman J.,79,199,655,239,2.20,1.61,4-6 1-6
6,Brisbane International,2013-01-01,ATP250,Outdoor,Hard,1st Round,3,Falla A.,Levine J.,Falla A.,54,104,809,530,2.20,1.61,6-1 7-6
7,Brisbane International,2013-01-01,ATP250,Outdoor,Hard,1st Round,3,Kudla D.,Melzer J.,Melzer J.,137,29,402,1177,2.62,1.44,6-2 4-6 4-6
8,Brisbane International,2013-01-01,ATP250,Outdoor,Hard,1st Round,3,Robredo T.,Harrison R.,Robredo T.,114,69,495,710,3.00,1.36,6-4 7-6
9,Brisbane International,2013-01-01,ATP250,Outdoor,Hard,1st Round,3,Baker B.,Dimitrov G.,Dimitrov G.,61,48,756,866,3.00,1.36,3-6 6-7


In [7]:
rank_1_df = df[df['Rank_1'] == 1].sort_values('Date',ignore_index=True)
rank_1_df

,Tournament,Date,Series,Court,Surface,Round,Best of,Player_1,Player_2,Winner,Rank_1,Rank_2,Pts_1,Pts_2,Odd_1,Odd_2,score
0,China Open,2012-10-02,ATP500,Outdoor,Hard,1st Round,3,Djokovic N.,Rosol L.,Djokovic N.,1,46,11120,951,1.01,21.00,6-0 6-3
1,China Open,2012-10-07,ATP500,Outdoor,Hard,The Final,3,Djokovic N.,Nadal R.,Djokovic N.,1,2,11120,10860,1.72,2.10,6-3 6-4
2,Australian Open,2013-01-24,Grand Slam,Outdoor,Hard,Semifinals,5,Djokovic N.,Ferrer D.,Djokovic N.,1,5,12920,6505,1.08,8.00,6-2 6-2 6-1
3,Australian Open,2013-01-27,Grand Slam,Outdoor,Hard,The Final,5,Djokovic N.,Murray A.,Djokovic N.,1,3,12920,8000,1.53,2.62,6-7 7-6 6-3 6-2
4,Dubai Tennis Championships,2013-02-27,ATP500,Outdoor,Hard,2nd Round,3,Djokovic N.,Bautista R.,Djokovic N.,1,55,12960,788,1.01,26.00,6-1 7-6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
334,Dubai Tennis Championships,2023-03-02,ATP500,Outdoor,Hard,Quarterfinals,3,Djokovic N.,Hurkacz H.,Djokovic N.,1,11,6980,3065,1.10,7.00,6-3 7-5
335,Dubai Tennis Championships,2023-03-03,ATP500,Outdoor,Hard,Semifinals,3,Djokovic N.,Medvedev D.,Medvedev D.,1,7,6980,3320,1.44,2.75,4-6 4-6
336,Miami Open,2023-03-26,Masters 1000,Outdoor,Hard,3rd Round,3,Alcaraz C.,Lajovic D.,Alcaraz C.,1,76,7420,728,1.03,15.00,6-0 7-6
337,Miami Open,2023-03-28,Masters 1000,Outdoor,Hard,4th Round,3,Alcaraz C.,Paul T.,Alcaraz C.,1,19,7420,2045,1.13,6.00,6-4 6-4


In [18]:
fig = go.Figure(data=[go.Bar(x=rank_1_df['Player_1'], y=rank_1_df['Rank_1'])])

fig.update_layout(title="Gráfico de Barras")

pio.show(fig)

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed